In [1]:
import pandas as pd

In [3]:
data=pd.read_csv(r"C:\Users\Utkarsh\Downloads\Coding\Gen_AI\my_project\customer_support_system\data\amazon_product_review.csv")

In [4]:
data.head()

,product_id,product_title,rating,summary,review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i..."
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...


In [6]:
data.columns

Index(['product_id', 'product_title', 'rating', 'summary', 'review'], dtype='object')

In [8]:
data["review"][0]

"1-more flexible2-bass is very high3-sound clarity is good 4-battery back up to 6 to 8 hour's 5-main thing is fastest charging system is available in that. Only 20 min charge and get long up to 4 hours back up 6-killing look awesome 7-for gaming that product does not support 100% if you want for gaming then I'll recommend you please don't buy but you want for only music then this product is very well for you.. 8-no more wireless headphones are comparing with that headphones at this pric..."

In [10]:
data["rating"][:5]

0    5
1    5
2    5
3    5
4    5
Name: rating, dtype: int64

In [13]:
data["product_title"][:5]

0    BoAt Rockerz 235v2 with ASAP charging Version ...
1    BoAt Rockerz 235v2 with ASAP charging Version ...
2    BoAt Rockerz 235v2 with ASAP charging Version ...
3    BoAt Rockerz 235v2 with ASAP charging Version ...
4    BoAt Rockerz 235v2 with ASAP charging Version ...
Name: product_title, dtype: object

In [15]:
data["summary"][:5]

0    Terrific purchase
1    Terrific purchase
2               Super!
3               Super!
4    Terrific purchase
Name: summary, dtype: object

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [3]:
openai_model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")
openai_model.invoke("hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-04-17', 'safety_ratings': []}, id='run--6baba9db-3999-4b98-900b-933f3a306652-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 257, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 246}})

In [ ]:
from langchain_astradb import AstraDBVectorStore
from dotenv import load_dotenv
import os
import pandas as pd
from data_ingestion.data_transform import data_converter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")

os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["ASTRA_DB_API_ENDPOINT"]=ASTRA_DB_API_ENDPOINT
os.environ["ASTRA_DB_APPLICATION_TOKEN"]=ASTRA_DB_APPLICATION_TOKEN
os.environ["ASTRA_DB_KEYSPACE"]=ASTRA_DB_KEYSPACE

class ingest_data:
    
    def __init__(self):
        print("data ingestion class has init...")
        self.embeddings = GoogleGenerativeAIEmbeddings(model="model/text-embedding-004")
        self.data_converter=data_converter()

    def data_ingestion(self, status):
        vstore=AstraDBVectorStore(
            embedding=self.embeddings,
            collection_name="chatbotecomm",
            api_endpoint=ASTRA_DB_API_ENDPOINT,
            token=ASTRA_DB_APPLICATION_TOKEN,
            namespace=ASTRA_DB_KEYSPACE,
        )
        storage=status

        if storage==None:
            docs=self.data_converter.data_transformation()
            inserted_ids=vstore.add_documents(docs)
            print(inserted_ids)
        else:
            return vstore
        
        return vstore, inserted_ids

if __name__ == '__main__':
    ingest_data=ingest_data()
    vstore, inserted_ids=ingest_data.data_ingestion(None)
    print(f"\nInserted{len(inserted_ids)} documents.")
    # results=vstore.similarity_search('can you tell me the low budget headphone')
    # for res in results:
    #     print(f"{res.page_content} {res.metadata}")

In [ ]:
from langchain_astradb import AstraDBVectorStore
from dotenv import load_dotenv
import os
from pathlib import Path
from data_ingestion.data_transform import data_converter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# ✅ Load .env from two directories up
load_dotenv(dotenv_path=Path(__file__).resolve().parents[2] / '.env')

# ✅ Get environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")

# ✅ Ensure all variables are present
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment.")
if not ASTRA_DB_API_ENDPOINT:
    raise ValueError("ASTRA_DB_API_ENDPOINT not found in environment.")
if not ASTRA_DB_APPLICATION_TOKEN:
    raise ValueError("ASTRA_DB_APPLICATION_TOKEN not found in environment.")
if not ASTRA_DB_KEYSPACE:
    raise ValueError("ASTRA_DB_KEYSPACE not found in environment.")

# ✅ Set env vars for SDKs
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["ASTRA_DB_API_ENDPOINT"] = ASTRA_DB_API_ENDPOINT
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = ASTRA_DB_APPLICATION_TOKEN
os.environ["ASTRA_DB_KEYSPACE"] = ASTRA_DB_KEYSPACE

class ingest_data:

    def __init__(self):
        print("Data ingestion class initialized...")
        self.embeddings = GoogleGenerativeAIEmbeddings(model="model/text-embedding-001")
        self.data_converter = data_converter()

    def data_ingestion(self, status):
        vstore = AstraDBVectorStore(
            embedding=self.embeddings,
            collection_name="chatbotecomm",
            api_endpoint=ASTRA_DB_API_ENDPOINT,
            token=ASTRA_DB_APPLICATION_TOKEN,
            namespace=ASTRA_DB_KEYSPACE,
        )

        if status is None:
            docs = self.data_converter.data_transformation()
            inserted_ids = vstore.add_documents(docs)
            print(f"Inserted IDs: {inserted_ids}")
            return vstore, inserted_ids
        else:
            return vstore, []

if __name__ == '__main__':
    ingest = ingest_data()
    vstore, inserted_ids = ingest.data_ingestion(None)
    print(f"\n✅ Inserted {len(inserted_ids)} documents.")
